In [1]:
import pandas as pd
import numpy as np

## Initialization

Create the base network as a matrix of costs, where a cost of NaN represents a non-adjacent node. The cost to the same node (diagonal of the matrix) is also listed as NaN for programming purposes. Typically, the diagonal should be listed as a 0, but this creates problems in the scripts for Djikstra's Algorithm and the Bellman-Ford Algorithm.

In [2]:
network = pd.DataFrame(
    [[np.nan,5,1,np.nan,np.nan,np.nan],
     [np.nan,np.nan,4,6,np.nan,np.nan],
     [np.nan,np.nan,np.nan,7,2,np.nan],
     [np.nan,np.nan,np.nan,np.nan,np.nan,2],
     [np.nan,1,np.nan,3,np.nan,8],
     [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]])

In [3]:
display(network)

,0,1,2,3,4,5
0,NaN,5.0,1.0,NaN,NaN,NaN
1,NaN,NaN,4.0,6.0,NaN,NaN
2,NaN,NaN,NaN,7.0,2.0,NaN
3,NaN,NaN,NaN,NaN,NaN,2.0
4,NaN,1.0,NaN,3.0,NaN,8.0
5,NaN,NaN,NaN,NaN,NaN,NaN


## Djikstra's Algorithm

In [4]:
# Initialize the temporary and permanent labels for optimal costs
permanent = pd.Series([0,np.nan,np.nan,np.nan,np.nan,np.nan])
temp = pd.Series([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

In [5]:
previousPermanentNodeIndex = 0
iteration = 1

while permanent.isnull().values.any():
    
    # Print iteration number
    print(f'Iteration {iteration} permanent costs')

    # Calculate the new possible node costs using the previous permanent node
    newNodeCosts = network.iloc[previousPermanentNodeIndex] + permanent[previousPermanentNodeIndex]

    # Find min between new possible node costs and temporary costs, call it calcCosts
    calcCosts = np.fmin(newNodeCosts, temp)

    # Find min of the calcCosts and the associated node, make the node
    # and the cost permanent
    newMin = np.nanmin(calcCosts)
    newMinIndex = np.argmin(calcCosts)

    # Update the permanent costs vector
    permanent[newMinIndex] = newMin
    print(permanent)

    # Update the temporary costs vector
    temp = np.fmin(calcCosts, temp)
    # Make sure that any permanent nodes are't included
    temp[permanent.notna()] = np.nan

    # Update the previousPermanentNodeIndex
    previousPermanentNodeIndex = newMinIndex

    # Update iteration for display purposes
    iteration += 1

Iteration 1 permanent costs
0    0.0
1    NaN
2    1.0
3    NaN
4    NaN
5    NaN
dtype: float64
Iteration 2 permanent costs
0    0.0
1    NaN
2    1.0
3    NaN
4    3.0
5    NaN
dtype: float64
Iteration 3 permanent costs
0    0.0
1    4.0
2    1.0
3    NaN
4    3.0
5    NaN
dtype: float64
Iteration 4 permanent costs
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    NaN
dtype: float64
Iteration 5 permanent costs
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    8.0
dtype: float64


## Bellman-Ford Algorithm

In [6]:
# Initialize costs vector
nodeCosts = pd.Series([0, np.nan, np.nan, np.nan, np.nan, np.nan])
previousCosts = pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])

# Iterate the amount of nodes in the network
# While Bellman-Ford only needs V - 1 iterations, we run V 
for i in range(len(nodeCosts)):
    
    # Check for negative cycle, detected if i reaches the Vth iteration
    if i == len(nodeCosts):
        print("There was a negative cycle in the network")
    else:
        print(f"Iteration {i+1}")

    # For each iteration, iterate through all nodes to see if a smaller
    # cost can be found to a connecting/adjacent node
    for nodeIndex in range(len(nodeCosts)):
        checkCosts = network.iloc[nodeIndex] + nodeCosts[nodeIndex]
        nodeCosts = np.fmin(checkCosts, nodeCosts)
    print(nodeCosts)

    # Check if nodeCosts was updated at all or not
    # If not, the nodeCosts have converged and we can break the loop
    if pd.Series.equals(nodeCosts, previousCosts):
        print("The node costs have converged")
        break

    # Update the previous costs vector for the next iteration
    previousCosts = nodeCosts

    

Iteration 1
0     0.0
1     4.0
2     1.0
3     6.0
4     3.0
5    10.0
dtype: float64
Iteration 2
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    8.0
dtype: float64
Iteration 3
0    0.0
1    4.0
2    1.0
3    6.0
4    3.0
5    8.0
dtype: float64
The node costs have converged


## Floyd Algorithm

Reinitialize network for distance matrix so that it includes 0 on the diagonal of the matrix

In [7]:
distances = pd.DataFrame(
    [[0,5,1,np.nan,np.nan,np.nan],
     [np.nan,0,4,6,np.nan,np.nan],
     [np.nan,np.nan,0,7,2,np.nan],
     [np.nan,np.nan,np.nan,0,np.nan,2],
     [np.nan,1,np.nan,3,0,8],
     [np.nan,np.nan,np.nan,np.nan,np.nan,0]])

display(distances)

,0,1,2,3,4,5
0,0.0,5.0,1.0,NaN,NaN,NaN
1,NaN,0.0,4.0,6.0,NaN,NaN
2,NaN,NaN,0.0,7.0,2.0,NaN
3,NaN,NaN,NaN,0.0,NaN,2.0
4,NaN,1.0,NaN,3.0,0.0,8.0
5,NaN,NaN,NaN,NaN,NaN,0.0


In [8]:
# Iterate through each possible intermediate node
for intermediateNode in range(distances.shape[0]):
    # Iterate through each possible start node
    for startNode in range(distances.shape[0]):
        # Iterate through each possible end node
        for endNode in range(distances.shape[1]):
            # Update the distance if a shorter path is found
            distances.loc[startNode,endNode] = np.nanmin([distances.loc[startNode,endNode], distances.loc[startNode,intermediateNode] + distances.loc[intermediateNode,endNode]])

display(distances)


/var/folders/v_/lft8__tj6fd561d9kywt6m21d_dv91/T/ipykernel_48581/2146301407.py:8: RuntimeWarning: All-NaN axis encountered
  distances.loc[startNode,endNode] = np.nanmin([distances.loc[startNode,endNode], distances.loc[startNode,intermediateNode] + distances.loc[intermediateNode,endNode]])


,0,1,2,3,4,5
0,0.0,4.0,1.0,6.0,3.0,8.0
1,NaN,0.0,4.0,6.0,6.0,8.0
2,NaN,3.0,0.0,5.0,2.0,7.0
3,NaN,NaN,NaN,0.0,NaN,2.0
4,NaN,1.0,5.0,3.0,0.0,5.0
5,NaN,NaN,NaN,NaN,NaN,0.0
